## Instalamos las dependencias

Vamos a instalar los módulos de Python necesarios, utilizando pip. Si en tu ordenador la versión por defecto de Python es 2.7 y quieres instalarlos para python3, tendrás que sustituir pip por pip3.

In [ ]:
!pip install pandas geopandas
!pip install --pre cartoframes

## Obtenemos los datos
Vamos a descargarnos un csv con datos sobre las peticiones que realizan los ciudadanos al gobierno de Reino Unido.

Después los importaremos mediante la librería *Pandas*, la cual utilizaremos para analizarlos y obtener relaciones entre ellos. La estructura de datos principal en pandas es el *DataFrame*, con la que trabajaremos de forma similar a como lo haríamos con una hoja de cálculo, pero de manera mucho más eficiente.

In [ ]:
import pandas

In [ ]:
all_petitions = pandas.read_csv('UKData/petitions.csv')
all_petitions

## Juguemos con los datos
Si observamos la anterior tabla, vemos que cada fila representa una petición concreta al gobierno de UK. Las columnas contienen la información del número de personas que apoyan la petición en cada distrito electoral. 

¿Y qué hacemos con estos datos? Tal y como están es difícil obtener conclusiones... Para empezar, vamos a ver qué peticiones son más populares, es decir, cuáles son las que han obtenido mayor número de votos.

In [ ]:
# Añadimos una nueva columna, el total de votos
all_petitions['total'] = all_petitions.iloc[:, 2:].sum(axis=1)

In [ ]:
# Ordenamos los datos en función de dicha columna y nos quedamos con los primeros 100 
top_petitions = all_petitions.sort_values('total', ascending=False)[:100]
top_petitions

Parece que lo que más le importa a esta gente son los referéndums y que Donald Trump no les haga una visita...

¿Qué otra cosas serán las que más les preocupen a los detractores de Donald Trump?

In [ ]:
# Hallamos la correlación
top_petitions.corr()

Hemos calculado la correlación de las columnas... pero las columnas definen los distritos electorales y no es eso lo que queremos. Necesitamos calcular la correlación entre las filas, que representan las peticiones (o acciones). 

¿Qué operación matricial podemos aplicar a la tabla?

In [ ]:
top_petitions.T

In [ ]:
top_petitions.T.corr()

¿Qué ha pasado? Pues que estamos intentando calcular correlaciones con campos de texto... (los nombres de las acciones)

Vamos a probar a tomar de la tabla sólo las columnas que contienen los datos relevantes y repetimos el proceso de trasponer y correlar...

In [ ]:
data_columns = top_petitions.columns.drop(['id', 'total', 'action'])
top_petitions.loc[:, data_columns].T.corr()

¿Qué son 122, 976, 534...? Son identificadores de las distintas acciones, pero ¿no sería mejor ver el texto de la acción en su lugar?

Para ello, podemos marcar la columna *action* como cabecera de la tabla

In [ ]:
top_petitions.set_index('action').loc[:, data_columns].T.corr()

Bueno, ¡pues parece que tiene buena pinta! Ahora que calculamos correctamente las correlaciones, vamos a guardarlas en un nuevo DataFrame y sólo nos queda ordenarlas para saber qué otras peticiones son las que más les interesan a los detractores de Donald Trump...

In [ ]:
correlation_df = top_petitions.set_index('action').loc[:, data_columns].T.corr()
correlation_df.sort_values('Block Donald J Trump from UK entry', ascending=False)

## Ahora, ¡a pintar mapas!

Primero vamos a probar *Geopandas*, una extensión de pandas que nos permite trabajar con datos geográficos de manera más sencilla.

Empezaremos por obtener las regiones de Reino Unido.

In [ ]:
import geopandas

In [ ]:
boundaries = geopandas.read_file('UKData/uk_boundaries.geojson')
boundaries

La variable `boundaries` no es más que un DataFrame con la información del polígono que delimita cada región o distrito electoral (si nos fijamos, los identificadores de la columna *geoid* coinciden con los que teníamos en la tabla de peticiones).

Vamos a pintarlo utilizando *Matplotlib*:

In [ ]:
# Directiva de Jupyter para pintar "inline" el objeto que devuelve matplotlib
%matplotlib inline    
boundaries.plot()

In [ ]:
# No se ve un pijo, ¡más grande!
boundaries.plot(figsize=(10, 10))

## Llegó el momento publicitario...

Vamos a utilizar la librería *Cartoframes* para subir los datos a CARTO, visualizarlos y trabajar con ellos en la plataforma. 

¿Y por qué vamos a usar CARTO? Pues aparte de porque nos da de comer a la familia, porque permite mostrar el mapa en una interfaz web interactiva (y hacerlo público fácilmente), además de permitir realizar análisis más interesantes que los ofrecidos por geopandas, cómo veremos más adelante.

En primer lugar, necesitamos obtener nuestro API Key de la interfaz web de CARTO. Una vez logueados, al clickar en el avatar aparece un menú con la opción *Your API Keys*, que contiene lo que necesitamos.

In [ ]:
import cartoframes
CF = cartoframes.CartoContext(
    creds=cartoframes.Credentials(username='pycones01', key='1234567890123456789012345678901234567890')
)

Vamos a subir nuestro DataFrame de regiones de UK (`boundaries`) a CARTO y crear un mapa con dos capas: la capa del mapa base y una segunda capa con estos datos.

In [ ]:
CF.write(boundaries, 'uk_boundaries', overwrite=True)

In [ ]:
CF.map(layers=[
    cartoframes.BaseMap('light'),
    cartoframes.Layer('uk_boundaries'),
], interactive=True)

## Mostremos nuestros datos

Hasta ahora, nos hemos limitado a mostrar datos de regiones de Reino Unido, pero lo que realmente queremos es mostrar los datos de las peticiones. Para ello, tenemos que relacionar nuestras dos tablas: `boundaries`, que contiene los datos geográficos de las distintas regiones, y `top_petitions`, que contiene los datos numéricos (votos de las peticiones en cada región).

Podemos conseguir esto utilizando la función `merge` de pandas. Para usar merge, debemos tener una columna en común entre ambas tablas, en este caso *geoid*. Pero en la tabla de peticiones, no tenemos tal columna, si no que es la cabecera de las filas. Debemos por tanto transponer esta tabla una vez más, de forma que la disposición sea la correcta.

A continuación, indicamos que queremos hacer corresponder la columna *geoid* de la tabla de la izquierda (`boundaries`) con la cabecera o *index* de la tabla de la derecha (`top_petitions`).

In [ ]:
geo_petitions = boundaries.merge(top_petitions.set_index('action').T, left_on='geoid', right_index=True)
geo_petitions

Ahora que hemos relacionado las tablas y tenemos un DataFrame que contiene tanto la información del número de votantes como la geometría de la región, vamos a pintarlo en un mapa con geopandas:

In [ ]:
geo_petitions.plot('Block Donald J Trump from UK entry', figsize=(10, 10))

También podemos hacer lo mismo en CARTO. En este caso, subimos la tabla de unión (`geo_petitions`), pero sólo con las columnas que nos interesan, para ahorrar ancho de banda :)

In [ ]:
trump = geo_petitions[['Block Donald J Trump from UK entry', 'geometry', 'geoid']]
# Renombramos las columnas para escribir menos...
trump.columns = ['trump_haters', 'geometry', 'geoid']
CF.write(trump, 'trump', overwrite=True)

In [ ]:
CF.map(layers=[
    cartoframes.BaseMap('light'),
    cartoframes.Layer('trump',
        color={'column': 'trump_haters',
               'scheme': cartoframes.styling.sunset(5)}),
], interactive=True)

Si nos fijamos bien, lo que hemos conseguido realmente se parece mucho a un mapa de densidad de población, es decir, muchos más votantes en las ciudades que en las zonas rurales.

Esto es porque hemos pintado el número total de votantes, que evidentemente será mayor en las ciudades por el mero hecho de que allí vive más gente.

Vamos a intentar normalizar los datos, es decir, dividirlos por la población total, para hallar el porcentaje de votantes, que es una medida más significativa.

## Normalización por área

Antes de nada, con geopandas es muy fácil normalizar los datos del número de votantes por el tamaño de cada región. No es la mejor de las normalizaciones, pero es rápido de ver, así que merece la pena hacer el experimento.

Vamos a añadir una nueva columna a nuestro DataFrame, *haters_per_area*, aprovechándonos de la propiedad *area* de geopandas:

In [ ]:
geo_petitions['haters_per_area'] = geo_petitions['Block Donald J Trump from UK entry'] / geo_petitions.area
geo_petitions.plot('haters_per_area', figsize=(10,10))

Uhm... Aún más concentrado en las ciudades...

## Normalización por población

Con cartoframes es muy sencillo normalizar por población gracias al *Data Observatory*, un repositorio de datos que contiene, entre otros muchos, datos de la población de Reino Unido.

Vamos al catálogo del repositorio: https://cartodb.github.io/bigmetadata y ahí buscamos cuál es el identificador de los datos de población de Reino Unido (pista: es "uk.ons.LC2102EW0001").  

Con este identificador, llamamos a la función `data_augment` para añadir una nueva columna con la población a la tabla `trump` que habíamos creado previamente en CARTO.

In [ ]:
uk_pop = [{'numer_id': 'uk.ons.LC2102EW0001', 'normalization': 'prenormalized'}]
augmented = CF.data_augment('trump', uk_pop)
augmented

In [ ]:
# Calculamos el porcentaje de detractores de Trump por región, teniendo en cuenta el número de votos y la población
# de dicha región
augmented['trump_hater_percent'] = augmented['trump_haters'] / augmented['total_pop_prenormalized_2011']
augmented

In [ ]:
CF.write(augmented, 'trump_percent', overwrite=True)

In [ ]:
CF.map(layers=[
    cartoframes.BaseMap('light', labels='front'),
    cartoframes.Layer('trump_percent',
        color={'column': 'trump_hater_percent',
               'scheme': cartoframes.styling.burg(5)}),
], interactive=True, size=(900, 600))

Dado que hemos normalizado por población, cabría esperar ver un mapa con un color similar en todas sus áreas si el apoyo de la propuesta fuera uniforme...

Sin embargo, podemos observar una mayor concentración de votos en las ciudades respecto a las áreas rurales, con lo que los detractores de Trump se concentran en los núcleos importantes de población (sobre todo, en Londres).

## Extra: Normalizar por población al principio

Repitamos el análisis, normalizando por población desde el principio, para calcular la correlación sobre el % de votos de cada petición (en vez del número total).

In [ ]:
augmented_boundaries = boundaries.merge(augmented).set_index('geoid')
augmented_boundaries

In [ ]:
normalized_top_petitions = top_petitions.copy()
for col in data_columns:
    normalized_top_petitions[col] /= augmented_boundaries.loc[col, 'total_pop_prenormalized_2011']
normalized_top_petitions

In [ ]:
norm_correlation_matrix = normalized_top_petitions.set_index('action').loc[:, data_columns].T.corr()
norm_correlation_matrix.sort_values('Block Donald J Trump from UK entry', ascending=False)

## Extra: Sobre proyecciones y áreas

Cuando calculamos el área, ésta se calcula en función de la proyección que estamos utilizando. Una proyección es una manera de traducir el esferoide terrestre a un plano bidimensional.

Las más conocidas son:
- WSG84 (EPSG: 4326). Las típicas coordenadas latitud/longitud.
- Mercator (EPSG: 3395). La que estamos acostumbrados a ver en mapas.

En el primer caso, el área se daría en grados cuadrados y, en el segundo, las unidades son arbitrarias (similares a metros, pero sólo precisa cerca del ecuador). También existen proyecciones específicas para calcular áreas, como la que usamos en este ejemplo.

Quizás más interesante que ver el efecto sobre el área, es verlo sobre el mapa. Según la proyección, algunas partes del mapa parecerán estiradas o encogidas.

In [ ]:
geo_petitions.plot('haters_per_area', figsize=(10,10))

geo_petitions.crs = {'init': 'epsg:4326'}
mercator = geo_petitions.to_crs({'init': 'epsg:3395'})
mercator.plot('haters_per_area', figsize=(10,10))

equalarea = geo_petitions.to_crs({'init': 'epsg:23090'})
equalarea.plot('haters_per_area', figsize=(10,10))

geo_petitions.area[1], mercator.area[1], equalarea.area[1]